In [7]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests as req, zipfile, io
import json
import quandl as qdl
import configparser
import time
import datetime

config = configparser.ConfigParser()
config.read('../dl.cfg')

qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']

In [15]:
def getTickers(file_path):
    df = pd.read_csv(file_path)
    tickers = df.iloc[:,0].tolist()
    return tickers

tickers = getTickers("../data/WIKI_metadata.csv")

## AAII Investor Sentiment

In [16]:
investor_sentiment = qdl.get("AAII/AAII_SENTIMENT")

In [27]:
filter_sp = investor_sentiment.columns[-3:]

filtered_sentiment = investor_sentiment.iloc[:, :-3]
filtered_sentiment.head()

,Bullish,Neutral,Bearish,Total,Bullish 8-Week Mov Avg,Bull-Bear Spread,Bullish Average,Bullish Average + St. Dev,Bullish Average - St. Dev
Date,,,,,,,,,
1987-06-26,NaN,NaN,NaN,NaN,NaN,NaN,0.386014,0.489178,0.28285
1987-07-17,NaN,NaN,NaN,NaN,NaN,NaN,0.386014,0.489178,0.28285
1987-07-24,0.36,0.50,0.14,1.0,NaN,0.22,0.386014,0.489178,0.28285
1987-07-31,0.26,0.48,0.26,1.0,NaN,0.00,0.386014,0.489178,0.28285
1987-08-07,0.56,0.15,0.29,1.0,NaN,0.27,0.386014,0.489178,0.28285


In [44]:
def get_investor_sentiment():
    qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    raw_sentiments = qdl.get("AAII/AAII_SENTIMENT", paginate=True)
    filtered_sentiments = raw_sentiments.iloc[:, :-3]
    finished_sentiments = filtered_sentiments.reset_index()
    return finished_sentiments

In [45]:
df = get_investor_sentiment()
df.head()

,Date,Bullish,Neutral,Bearish,Total,Bullish 8-Week Mov Avg,Bull-Bear Spread,Bullish Average,Bullish Average + St. Dev,Bullish Average - St. Dev
0,1987-06-26,NaN,NaN,NaN,NaN,NaN,NaN,0.386014,0.489178,0.28285
1,1987-07-17,NaN,NaN,NaN,NaN,NaN,NaN,0.386014,0.489178,0.28285
2,1987-07-24,0.36,0.50,0.14,1.0,NaN,0.22,0.386014,0.489178,0.28285
3,1987-07-31,0.26,0.48,0.26,1.0,NaN,0.00,0.386014,0.489178,0.28285
4,1987-08-07,0.56,0.15,0.29,1.0,NaN,0.27,0.386014,0.489178,0.28285


## Yale S&P Composite Data

In [31]:
raw_sp_comp = qdl.get("YALE/SPCOMP", paginate=True)

In [34]:
raw_sp_comp.head()

,S&P Composite,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
1871-01-31,4.44,0.26,0.4,12.464061,5.320000,91.371503,5.350584,8.231667,NaN
1871-02-28,4.50,0.26,0.4,12.844641,5.323333,89.862377,5.192048,7.987767,NaN
1871-03-31,4.61,0.26,0.4,13.034972,5.326667,90.714810,5.116237,7.871133,NaN
1871-04-30,4.74,0.26,0.4,12.559226,5.330000,96.806121,5.310040,8.169293,NaN
1871-05-31,4.86,0.26,0.4,12.273812,5.333333,101.565027,5.433520,8.359261,NaN


In [47]:
def get_yale_sp_comp_data():
    qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    raw_sp_comp = qdl.get("YALE/SPCOMP", paginate=True)
    finished_data = raw_sp_comp.reset_index()
    return finished_data

## Yale Individual Confidence

## US Stock Market Confidence Indices, Individual (Source: Yale Department of Economics, Quandl Free API)
    * Date: Timestamp - (Year-Month-Day, non-nullable).
    * Valuation_Indices: Double - (nullable) - Confidence in Stock Market. 
    * Valuation_indices_Std_Err: Double - (nullable) - Standard error of Confidence Indices.
    * Crash_confidence: Double - (nullable) - Crash Confidence in Stock Market.
    * Crash_confidence_Std_Err: Double - (nullable) - CC standard error.
    * Buy_on_Dips: Double - (nullable) - Confidence to buy during dips.
    * Buy_on_Dips_Std_Err: Double - (nullable) - Standard error of BoD. 

In [49]:
raw_valuation_conf = qdl.get("YALE/US_CONF_INDEX_VAL_INDIV", paginate=True).reset_index()
raw_crash_conf = qdl.get("YALE/US_CONF_INDEX_CRASH_INDIV", paginate=True).reset_index()
raw_buy_dips = qdl.get("YALE/US_CONF_INDEX_BUY_INDIV", paginate=True).reset_index()

In [51]:
raw_valuation_conf.columns

Index(['Date', 'Index Value', 'Standard Error'], dtype='object')

In [52]:
val_conf = raw_valuation_conf.rename(columns={"Index Value": "Index_Value_Valuation_Confidence", "Standard Error": "Standard_Error_Valuation"})
crash_conf = raw_crash_conf.rename(columns={"Index Value": "Index_Value_Crash_Confidence", "Standard Error": "Standard_Error_Crash"})
bod_conf = raw_buy_dips.rename(columns={"Index Value": "Index_Value_Buy_on_Dip_Confidence", "Standard Error": "Standard_Error_Buy_on_Dip"})

In [53]:
val_conf.columns

Index(['Date', 'Index_Value_Valuation_Confidence', 'Standard_Error_Valuation'], dtype='object')

In [56]:
confidence_ = pd.merge(val_conf, crash_conf, on="Date")

In [58]:
confidence_final = pd.merge(confidence_, bod_conf, on="Date")

In [59]:
confidence_final.head()

,Date,Index_Value_Valuation_Confidence,Standard_Error_Valuation,Index_Value_Crash_Confidence,Standard_Error_Crash,Index_Value_Buy_on_Dip_Confidence,Standard_Error_Buy_on_Dip
0,1989-10-31,73.27,4.40,41.59,4.64,41.76,5.17
1,1996-10-31,51.97,4.43,33.08,4.13,53.04,4.65
2,1999-04-30,31.61,3.73,34.59,3.77,65.49,3.99
3,1999-10-31,34.83,5.05,32.29,4.77,53.42,5.84
4,2000-04-30,27.68,4.23,30.25,4.21,60.78,4.83


In [ ]:
def confidence_data():
    qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    
    # Make API call to get Confidence Tables:
    
    raw_valuation_conf = qdl.get("YALE/US_CONF_INDEX_VAL_INDIV", paginate=True).reset_index()
    raw_crash_conf = qdl.get("YALE/US_CONF_INDEX_CRASH_INDIV", paginate=True).reset_index()
    raw_buy_dips = qdl.get("YALE/US_CONF_INDEX_BUY_INDIV", paginate=True).reset_index()
    
    # Rename the Columns of Each Table:
    val_conf = raw_valuation_conf.rename(columns={"Index Value": "Index_Value_Valuation_Confidence", "Standard Error": "Standard_Error_Valuation"})
    crash_conf = raw_crash_conf.rename(columns={"Index Value": "Index_Value_Crash_Confidence", "Standard Error": "Standard_Error_Crash"})
    bod_conf = raw_buy_dips.rename(columns={"Index Value": "Index_Value_Buy_on_Dip_Confidence", "Standard Error": "Standard_Error_Buy_on_Dip"})
    
    #Merge Tables:
    confidence_ = pd.merge(val_conf, crash_conf, on="Date")
    confidence_final = pd.merge(confidence_, bod_conf, on="Date")
    
    #return final table:
    return confidence_final

## Ticker Multi Single Day History

In [62]:
temp_url="https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol=AAPL,MSFT&date=2018-01-02&api_token="

In [283]:
ticker = 'MSFT'
ticker2 = '^XTJ'
ticker3 = 'AAPL'
ticker4 = 'PFG'
wt_api_key = config['WTD']['wtd_key']
date = '2018-01-02'

url1 = (f"https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol={ticker},{ticker2}&date={date}&api_token={wt_api_key}")
url2 = (f"https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol={ticker3},{ticker4}&date={date}&api_token={wt_api_key}")

In [284]:
print(url1)
print(url2)

https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol=MSFT,^XTJ&date=2018-01-02&api_token=VuZ6i10kmCH8oVGvFiG3QWLiSrQeIbnAT9Y15U6MBygT9ypMxgOsn6oquVqU
https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol=AAPL,PFG&date=2018-01-02&api_token=VuZ6i10kmCH8oVGvFiG3QWLiSrQeIbnAT9Y15U6MBygT9ypMxgOsn6oquVqU


In [310]:
request1 = req.get(url1).json()
request2 = req.get(url2).json()

In [311]:
print(json.dumps(request1))
print(json.dumps(request2))

{"date": "2018-01-02", "data": {"MSFT": {"open": "86.13", "close": "85.95", "high": "86.31", "low": "85.50", "volume": "22483797"}, "^XTJ": {"open": "1307.80", "close": "1316.80", "high": "1320.20", "low": "1307.50", "volume": "0"}}}
{"date": "2018-01-02", "data": {"AAPL": {"open": "170.16", "close": "172.26", "high": "172.30", "low": "169.26", "volume": "25555934"}, "PFG": {"open": "71.18", "close": "70.38", "high": "71.32", "low": "69.92", "volume": "760256"}}}


In [312]:
newdf1 = pd.DataFrame(request1)
newdf2 = pd.DataFrame(request2)

In [290]:
newdf1.head()

,date,data
MSFT,2018-01-02,"{'open': '86.13', 'close': '85.95', 'high': '8..."
^XTJ,2018-01-02,"{'open': '1307.80', 'close': '1316.80', 'high'..."


In [291]:
newdf1.append(newdf2)

,date,data
MSFT,2018-01-02,"{'open': '86.13', 'close': '85.95', 'high': '8..."
^XTJ,2018-01-02,"{'open': '1307.80', 'close': '1316.80', 'high'..."
AAPL,2018-01-02,"{'open': '170.16', 'close': '172.26', 'high': ..."
PFG,2018-01-02,"{'open': '71.18', 'close': '70.38', 'high': '7..."


In [122]:
# lst = list(request['data'].keys())
# lst[0]

'AAPL'

In [296]:
new_df = pd.concat([newdf1.drop(['data'], axis=1), newdf1['data'].apply(pd.Series)], axis=1).reset_index()

In [297]:
new_df.head()

,date,open,close,high,low,volume
MSFT,2018-01-02,86.13,85.95,86.31,85.50,22483797
^XTJ,2018-01-02,1307.80,1316.80,1320.20,1307.50,0


In [293]:
ticker_df = pd.read_csv('../data/WIKI_metadata.csv')
ticker_lst = ticker_df['code']

In [161]:
def get_ticker_data():
    #Input API Keys
    wt_api_key = config['WTD']['wtd_key']
    qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    
    #Make list of ticker symbols, convert to string
    ticker_df = pd.read_csv('../data/WIKI_metadata.csv')
    ticker_lst = ticker_df['code']
    ticker_str = ','.join(ticker_lst)
    
    #Get list of Dates, utilize quandl API for specific dates needed
    investor_sentiment_raw = qdl.get("AAII/AAII_SENTIMENT").reset_index()
    dates = investor_sentiment_raw['Date']
    date_lst = [date.strftime('%Y-%m-%d') for date in dates]
    
    #setup df
    final_df = pd.DataFrame()
    
    #For each date in dates, get price data for each ticker
    for date in dates:
        url = (f"https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol={ticker_str}&date={date}&api_token={wt_api_key}")
        request = req.get(url)
        raw_df = pd.DataFrame(request)
        processed_df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1).reset_index()
        final_df.append(processed_df)
        
    #return dataframe
    return final_df

In [300]:
def get_ticker_data():
    #Input API Keys
    wt_api_key = config['WTD']['wtd_key']
    qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    
    #Make list of ticker symbols
    ticker_df = pd.read_csv('../data/WIKI_metadata.csv')
    ticker_lst = ticker_df['code']
    
    #Get List of Dates, utilize quandl API for specific dtes needed
    date_raw = qdl.get("AAII/AAII_SENTIMENT").reset_index()
    dates = date_raw['Date']
    date_lst = [date.strftime('%Y-%m-%d') for date in dates]
    
    raw_full_df = pd.DataFrame()
    
    #Loop through tickers and dates, append to dataframe
    for date in date_lst:
        for ticker in ticker_lst:
            url = (f"https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol={ticker}&date={date}&api_token={wt_api_key}")
            request = req.get(url).json()
            raw_df = pd.DataFrame(request)
            raw_full_df.append(raw_df)
            
    #Return a new processed dataframe
    processed_df = pd.concat([raw_full_df.drop(['data'], axis=1), raw_full_df['data'].apply(pd.Series)], axis=1).reset_index()
    
    return processed_df
    

In [302]:
wt_api_key = config['WTD']['wtd_key']
qdl.ApiConfig.api_key = config['QUANDL']['quandl_key']
    
    #Make list of ticker symbols
ticker_df = pd.read_csv('../data/WIKI_metadata.csv')
ticker_lst = ticker_df['code']
    
    #Get List of Dates, utilize quandl API for specific dtes needed
date_raw = qdl.get("AAII/AAII_SENTIMENT").reset_index()
dates = date_raw['Date']
date_lst = [date.strftime('%Y-%m-%d') for date in dates]
    
raw_full_df = pd.DataFrame()

In [ ]:
for date in date_lst:
    for ticker in ticker_lst:
        url = (f"https://api.worldtradingdata.com/api/v1/history_multi_single_day?symbol={ticker}&date={date}&api_token={wt_api_key}")
        request = req.get(url).json()
        raw_df = pd.DataFrame(request, columns=[0])
        raw_full_df.append(raw_df)

## Quick test to see if dataframe to redshift load works. 

In [9]:
def loadQuandl_yaleConf(conn, redshift_table):
    """
    Function that goes out, pulls indivdiual confidence data for valuation, crashes, and buy on dips. Combines into dataframe, and uploads to Redshift
    
    Parameters:
    
    conn (Database Connection): A connection to the Redshift Database.
    
    redshift_table (str): The name of the table we are loading data too.
    
    **kwargs (kwargs): Needed for Python Operator in Airflow to set parameters.
    
    Returns:
    None
    
    """
    #log.info('Beginning implementation of Quandl-Redshift Load for Yale S&P Data.')
    
    #Make API call to create dataframes with confidence index data on valuations, crashes, and buy on dips. 
    raw_valuation_conf = qdl.get("YALE/US_CONF_INDEX_VAL_INDIV", paginate=True).reset_index()
    raw_crash_conf = qdl.get("YALE/US_CONF_INDEX_CRASH_INDIV", paginate=True).reset_index()
    raw_buy_dips = qdl.get("YALE/US_CONF_INDEX_BUY_INDIV", paginate=True).reset_index()
    
    #Map columns of dataframes to new names for better data clarity.
    val_conf = raw_valuation_conf.rename(columns={"Index Value": "Index_Value_Valuation_Confidence", "Standard Error": "Standard_Error_Valuation"})
    crash_conf = raw_crash_conf.rename(columns={"Index Value": "Index_Value_Crash_Confidence", "Standard Error": "Standard_Error_Crash"})
    bod_conf = raw_buy_dips.rename(columns={"Index Value": "Index_Value_Buy_on_Dip_Confidence", "Standard Error": "Standard_Error_Buy_on_Dip"})

    #Merge data into one final dataframe.
    confidence_ = pd.merge(val_conf, crash_conf, on="Date")
    confidence_final = pd.merge(confidence_, bod_conf, on="Date")
    
    #Send to Redshift
    confidence_final.to_sql(redshift_table, conn, index=False, if_exists='replace')
    #log.info(f"Successfully loaded Individual's Confidence data to {table}.")

In [10]:
from sqlalchemy import create_engine
conn = create_engine("")

In [11]:
loadQuandl_yaleConf(conn, "us_stock_market_confidence_indices")